In [1]:
import torch.nn as nn
import torch
import copy

In [2]:
# from diffusers import UNet2DConditionModel
import sys
p = "src/"
sys.path.append(p)
from src.unet_2d_condition import UNet2DConditionModel
from transformers import AutoTokenizer, CLIPTextModel

unet = UNet2DConditionModel.from_pretrained("stabilityai/sd-turbo", subfolder="unet").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("stabilityai/sd-turbo", subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained("stabilityai/sd-turbo", subfolder="text_encoder").cuda()


/media/workspace/richamishra/miniconda3/envs/img2img-turbo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prompt = "John Oliver in a talk show"
caption_tokens = tokenizer(prompt, max_length=tokenizer.model_max_length,
                                            padding="max_length", truncation=True, return_tensors="pt").input_ids.cuda()
caption_enc = text_encoder(caption_tokens, )[0]

In [ ]:
class RefConv(torch.nn.Module):
    def __init__(self, convin_pretrained, convin_curr, ):
        super(RefConv, self).__init__()
        self.conv_in_pretrained = copy.deepcopy(convin_pretrained)
        self.conv_in_curr = copy.deepcopy(convin_curr)

    def forward(self, x, is_reference=False):
        import pdb; pdb.set_trace()
        if is_reference:
            x = self.conv_in_pretrained(x).detach()
        else:
            x = self.conv_in_curr(x)
        return x

In [ ]:
#convin_pretrained = copy.deepcopy(unet.conv_in)
#unet.conv_in = RefConv(convin_pretrained, unet.conv_in)

In [6]:
x = torch.randn((2,4,64,64)).cuda()
timesteps = torch.tensor([999], device="cuda").long()
unet(x, timesteps, encoder_hidden_states=caption_enc, is_reference=False)

RuntimeError: The size of tensor a (8192) must match the size of tensor b (4096) at non-singleton dimension 1